# Summarize melt model outputs for partner basins

In [1]:
from __future__ import print_function
%pylab notebook
import datetime as dt
import glob
import matplotlib.pyplot as plt
import pdb
from charistools.hypsometry import Hypsometry
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
from charistools.timeSeries import TimeSeries
import pandas as pd
import re
import os

Populating the interactive namespace from numpy and matplotlib


# Given a basin name, read the 3 melt output files

In [10]:
dir = '/scratch/summit/brodzik/ti_model/derived_hypsometries/partner_basins'
dir = '/pl/active/charis/ti_model/derived_hypsometries/REECv0'

In [11]:
%cd $dir
%ls

/pl/active/charis/ti_model/derived_hypsometries/REECv0
00area_by_elevation/             BR_OBJECTID9/
AM_OBJECTID100/                  GA_Karnali_at_Benighat/
AM_OBJECTID62/                   GA_Narayani_at_Devghat/
AM_OBJECTID64/                   GA_SaptaKosi_at_Chatara/
AM_OBJECTID65/                   GA_v01_OBJECTID1/
AM_OBJECTID66/                   GA_v01_OBJECTID10/
AM_OBJECTID68/                   GA_v01_OBJECTID12/
AM_OBJECTID69/                   GA_v01_OBJECTID2/
AM_OBJECTID72/                   GA_v01_OBJECTID22/
AM_OBJECTID77/                   GA_v01_OBJECTID3/
AM_OBJECTID78/                   GA_v01_OBJECTID32/
AM_OBJECTID79/                   GA_v01_OBJECTID4/
AM_OBJECTID80/                   GA_v01_OBJECTID6/
AM_OBJECTID81/                   GA_v01_OBJECTID9/
AM_OBJECTID82/                   IN_Hunza_at_DainyorBridge/
AM_OBJECTID83/                   IN_v01_OBJECTID0/
AM_OBJECTID84/                   IN_v01_OBJECTID10/
AM_OBJECTID85/                   IN_v01_OBJECTID1

In [15]:
def read_melt_for(basinName, dir):

    nstrikes = 2
    
    years = 2001 + np.arange(14)
    
    # Initialize an empty df
    index = [pd.to_datetime(year, format='%Y') for year in years]
        
    df = pd.DataFrame(index=index)
    df['EGImelt'] = 0.
    df['SOImelt'] = 0.
    df['SOLmelt'] = 0.
    
    for year in years:
        
        yearDt = pd.to_datetime(year, format='%Y')
        
        list = sorted(glob.glob("%s/%s/%s.%s.0100m.modicev04_%sstrike.*melt_by_elev*.txt" % (
            dir, basinName, basinName, str(year), str(nstrikes))))
        if len(list) != 3:
            raise ValueError("Unexpected number of melt files found for %s, %d." % (basinName, year))
        
        # Assume sorted list is EGI/SOI/SOL:
        egiMeltHyps = Hypsometry(list[0])
        df.loc[yearDt, 'EGImelt'] = egiMeltHyps.data_by_doy().sum()
        
        soiMeltHyps = Hypsometry(list[1])
        df.loc[yearDt, 'SOImelt'] = soiMeltHyps.data_by_doy().sum()
        
        solMeltHyps = Hypsometry(list[2])
        df.loc[yearDt, 'SOLmelt'] = solMeltHyps.data_by_doy().sum()
        

    return df


In [ ]:
%cd $dir
basinNames = sorted(glob.glob("SY_*_at_*"))
basinNames

In [13]:
basinNames = sorted(glob.glob("??_*_at_*"))
basinNames

['AM_Vakhsh_at_Komsomolabad',
 'AM_Vakhsh_at_Komsomolabad_copy',
 'GA_Karnali_at_Benighat',
 'GA_Narayani_at_Devghat',
 'GA_SaptaKosi_at_Chatara',
 'IN_Hunza_at_DainyorBridge',
 'SY_Naryn_at_NarynTown']

In [16]:
fig, ax = plt.subplots(len(basinNames), 1, figsize=(10, len(basinNames)*3))

for i, basinName in enumerate(basinNames):
    df = read_melt_for(basinName, dir)
    if len(basinNames) == 1:
        df.plot(ax=ax)
        ax.set_title(basinName)
    else:
        df.plot(ax=ax[i])
        ax[i].set_title(basinName)
    
fig.tight_layout()

<IPython.core.display.Javascript object>

ValueError: Unexpected number of melt files found for AM_Vakhsh_at_Komsomolabad_copy, 2001.